# PitViper Notebook Report

In [1]:
from functions_pitviper_nb import * 
import time
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [2]:
# Parameters
mageck_mle_outputs = "../../results/development/MAGeCK_MLE/"
mageck_rra_outputs = "../../results/development/MAGeCK_RRA/"
bagel_outputs = "../../results/development/BAGEL/"
crisphiermix_outputs = "../../results/development/CRISPhieRmix/"

In [3]:
results_path = [mageck_mle_outputs, mageck_rra_outputs, bagel_outputs, crisphiermix_outputs]

results = create_results_pitviper(results_path)

## Quality Control

In [32]:

def sgRNA_accross_conditions(result):
    conditions = [{'baseline':condition.split('_vs_')[1] , 'treatment':condition.split('_vs_')[0]} for condition in result.comparisons_dict.keys()]
    baselines=set([condition['baseline'] for condition in conditions])
    features=widgets.Text(value='MYC', placeholder='Feature to show...', description='Feature:')
    @interact(feature=features, baseline=baselines)
    def sgRNA_accross_conditions(feature, baseline):
        treatments = [condition['treatment'] for condition in conditions if condition['baseline'] == baseline]
        print('Feature:', feature)
        print('Baseline', baseline)
        print('Treatments', treatments)
        
        points = treatments + [baseline]
        
        print(points)
        
#         tables_to_concatenate = []
#         for point in points:
#             table = result.comparisons_dict[point]['table'].copy()
#             trt = comparison.split('_vs_')[0]
#             new_names = [(i,trt + "_" + i) for i in table.iloc[:, 1:].columns.values]
#             table.rename(columns = dict(new_names), inplace=True)
#             tables_to_concatenate.append(table)

#         result = pd.concat(tables_to_concatenate, axis=1)
#         result = result.iloc[:,~result.columns.duplicated()]
            


sgRNA_accross_conditions(results['MAGeCK_RRA'])

interactive(children=(Text(value='MYC', description='Feature:', placeholder='Feature to show...'), Dropdown(de…

## Gene essentiality analysis

### MAGeCK MLE - Feature accross conditions

In [5]:
feature_accros_conditions(results['MAGeCK_MLE'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J18', 'M07e-J4', 'M07e-J11'), value='M07…

### MAGeCK RRA - Feature accross conditions

In [6]:
feature_accros_conditions(results['MAGeCK_RRA'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J18', 'M07e-J4', 'M07e-J11'), value='M07…

### CRISPhieRmix - Feature accross conditions

In [7]:
feature_accros_conditions(results['CRISPhieRmix'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J18', 'M07e-J4', 'M07e-J11'), value='M07…

### BAGEL - Feature accross conditions

In [8]:
feature_accros_conditions(results['BAGEL'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J18', 'M07e-J4', 'M07e-J11'), value='M07…

## Annotations

### EnrichR

In [9]:
enrichr_plots(results)

interactive(children=(Dropdown(description='tool', options=('MAGeCK_MLE', 'MAGeCK_RRA', 'BAGEL', 'CRISPhieRmix…

In [10]:
# data = results['MAGeCK_MLE'].comparisons_dict['M07e-J25_vs_M07e-J4']['table']

def test_plotting(data):
    conditions = [{'baseline':condition.split('_vs_')[1] , 'treatment':condition.split('_vs_')[0]} for condition in data.comparisons_dict.keys()]
    baselines=set([condition['baseline'] for condition in conditions])
    @interact(baseline=baselines)
    def test_plotting(baseline):
        treatments = [condition['treatment'] for condition in conditions if condition['baseline'] == baseline]
        non_sig=widgets.ColorPicker(concise=False, description='Non-significant', value='#949494', disabled=False)
        sig=widgets.ColorPicker(concise=False, description='Significant', value='red', disabled=False)
        @interact(treatment=treatments, non_sig=non_sig, sig=sig)
        def test_plotting(treatment, non_sig, sig):
            print('Baseline:', baseline)
            print('Treatment:', treatment)
            comp = treatment + '_vs_' + baseline
            source = data.comparisons_dict[comp]['table']

            source['default_rank'] = source[treatment + '|beta'].rank()

            source.loc[source[treatment + '|fdr'] < 0.05, 'significant'] = 'Yes' 
            source.loc[source[treatment + '|fdr'] >= 0.05, 'significant'] = 'No'
            domain = ['Yes', 'No']
            range_ = [sig, non_sig]

            chart = alt.Chart(source).mark_circle(size=60).encode(
                x=alt.X('default_rank:Q'),
                y=alt.Y(treatment + '|beta:Q'),
                tooltip=['Gene', 'sgRNA', treatment + '|beta', treatment + '|fdr', 'significant'],
                color=alt.Color('significant', scale=alt.Scale(domain=domain, range=range_), legend=alt.Legend(title="Significativity:"))
            ).properties(width=700)

            line = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')

            chart = (chart + line)

            return chart

test_plotting(results['MAGeCK_MLE'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J18', 'M07e-J4', 'M07e-J11'), value='M07…